## ЭТАП I. ПОСТРОЕНИЕ МОДЕЛИ НА ОБУЧАЮЩЕЙ ВЫБОРКЕ - ЧАСТИ ИСТОРИЧЕСКОЙ ВЫБОРКИ И ЕЕ ПРОВЕРКА НА КОНТРОЛЬНОЙ ВЫБОРКЕ - ЧАСТИ ИСТОРИЧЕСКОЙ ВЫБОРКИ

In [1]:
# импортируем необходимые библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.stats import norm
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from pandas import Series, DataFrame
from collections import Counter, defaultdict

## I.1. Считывание CSV-файла, содержащего исторические данные, в объект DataFrame

In [2]:
# увеличиваем количество выводимых столбцов
pd.set_option('display.max_columns', 60)
# загружаем набор данных
data = pd.read_csv('Credit_OTP.csv', encoding='cp1251', sep=';')
# выводим первые 10 наблюдений
data.head(10)

,AGREEMENT_RK,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,EDUCATION,MARITAL_STATUS,GEN_INDUSTRY,GEN_TITLE,ORG_TP_STATE,ORG_TP_FCAPITAL,JOB_DIR,FAMILY_INCOME,PERSONAL_INCOME,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,TP_PROVINCE,REGION_NM,REG_FACT_FL,FACT_POST_FL,REG_POST_FL,REG_FACT_POST_FL,REG_FACT_POST_TP_FL,FL_PRESENCE_FL,OWN_AUTO,AUTO_RUS_FL,HS_PRESENCE_FL,COT_PRESENCE_FL,GAR_PRESENCE_FL,LAND_PRESENCE_FL,CREDIT,TERM,FST_PAYMENT,DL_DOCUMENT_FL,GPF_DOCUMENT_FL,FACT_LIVING_TERM,WORK_TIME,FACT_PHONE_FL,REG_PHONE_FL,GEN_PHONE_FL,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_NUM_PAYM,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED
0,59910150,0,49,1,0,1,2,1,Среднее специальное,Состою в браке,Торговля,Рабочий,Частная компания,Без участия,Вспомогательный техперсонал,от 10000 до 20000 руб.,5000,Оренбургская область,Оренбургская область,Оренбургская область,Оренбургская область,ПОВОЛЖСКИЙ,1,1,1,1,1,0,0,0,0,0,0,0,8000,6,8650,0,1,220,18.0,0,0,1,1,1,6,2,1,1580,1580,NaN
1,59910230,0,32,1,0,1,3,3,Среднее,Состою в браке,Торговля,Рабочий,Индивидуальный предприниматель,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,12000,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,Ставропольский край,ЮЖНЫЙ,1,1,1,1,0,0,0,0,0,0,0,0,21650,6,4000,0,1,137,97.0,1,0,1,1,1,6,1,1,4020,4020,NaN
2,59910525,0,52,1,0,1,4,0,Неполное среднее,Состою в браке,Информационные технологии,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,9000,Иркутская область,Иркутская область,Иркутская область,Иркутская область,ВОСТОЧНО-СИБИРСКИЙ,1,1,1,1,1,0,0,0,1,0,0,0,33126,12,4000,0,1,251,84.0,0,0,1,2,1,11,0,0,0,0,NaN
3,59910803,0,39,1,0,1,1,1,Высшее,Состою в браке,Образование,Руководитель среднего звена,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 20000 до 50000 руб.,25000,Ростовская область,Ростовская область,Ростовская область,Ростовская область,ЮЖНЫЙ,0,1,0,0,1,1,0,0,0,0,0,0,"8491,82",6,5000,0,0,36,168.0,1,1,1,1,1,6,3,1,"1589,923333",1590,NaN
4,59911781,0,30,1,0,0,0,0,Среднее,Состою в браке,Государственная служба,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,12000,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,Ставропольский край,ЮЖНЫЙ,1,1,1,1,0,0,0,0,1,0,0,0,21990,12,4000,0,1,83,101.0,1,0,1,2,1,16,2,1,"1152,15",2230,NaN
5,59911784,0,29,1,0,0,0,0,Среднее,Гражданский брак,Торговля,Специалист,Частная компания,Без участия,Адм-хоз. и трансп. службы,от 20000 до 50000 руб.,12000,Белгородская область,Белгородская область,Белгородская область,Белгородская область,ЦЕНТРАЛЬНЫЙ 1,1,1,1,1,1,0,1,0,0,0,0,0,30615,12,3500,0,0,108,40.0,1,0,1,2,1,11,0,0,0,0,NaN
6,59911832,0,35,1,0,0,1,1,Высшее,Разведен(а),Другие сферы,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 20000 до 50000 руб.,15000,Вологодская область,Вологодская область,Вологодская область,Вологодская область,ЦЕНТРАЛЬНЫЙ 2,1,1,1,1,1,1,0,0,0,0,0,0,10450,3,15000,0,1,48,6.0,0,0,1,2,2,14,0,0,0,0,NaN
7,59912034,0,41,1,0,1,0,0,Среднее,Состою в браке,Сельское хозяйство,Рабочий,Частная компания,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,6000,Волгоградская область,Волгоградская область,Волгоградская область,Волгоградская область,ЮЖНЫЙ,1,1,1,1,1,1,0,0,0,0,0,0,15035,10,465,0,1,204,6.0,1,1,1,2,1,11,0,0,0,0,NaN
8,59912560,0,53,1,0,0,2,0,Высшее,Состою в браке,Здравоохранение,Руководитель высшего звена,Частная компания,Без участия,Участие в основ. деятельности,свыше 50000 руб.,100000,Ярославская область,Ярославская область,Ярославская область,Ярославская область,ЦЕНТРАЛЬНЫЙ 2,1,1,1,1,1,0,0,0,0,0,0,0,"32914,31",4,"29080,92",0,0,228,156.0,1,0,1,1,1,3,0,0,0,0,NaN
9,59912659,0,43,1,0,1,0,0,Среднее специальное,Состою в браке,Образование,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,700

## I.2. Преобразование типов переменных

In [3]:
# выводим информацию о количестве непропущенных 
# наблюдений в переменных и типах переменных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15223 entries, 0 to 15222
Data columns (total 52 columns):
AGREEMENT_RK                  15223 non-null int64
TARGET                        15223 non-null int64
AGE                           15223 non-null int64
SOCSTATUS_WORK_FL             15223 non-null int64
SOCSTATUS_PENS_FL             15223 non-null int64
GENDER                        15223 non-null int64
CHILD_TOTAL                   15223 non-null int64
DEPENDANTS                    15223 non-null int64
EDUCATION                     15223 non-null object
MARITAL_STATUS                15223 non-null object
GEN_INDUSTRY                  13856 non-null object
GEN_TITLE                     13856 non-null object
ORG_TP_STATE                  13856 non-null object
ORG_TP_FCAPITAL               13858 non-null object
JOB_DIR                       13856 non-null object
FAMILY_INCOME                 15223 non-null object
PERSONAL_INCOME               15223 non-null object
REG_ADDRESS_PRO

In [4]:
# переменные, неверно записанные как количественные,
# преобразуем в тип object
object_columns = data.columns[data.columns.str.contains('FL|GENDER|OWN_AUTO|AGREEMENT')]
data[object_columns] = data[object_columns].astype('object')

In [5]:
# в указанных переменных заменяем запятую на точку в качестве
# десятичного разделителя и преобразуем в тип float
for column in ['PERSONAL_INCOME', 'CREDIT', 'FST_PAYMENT', 
               'LOAN_AVG_DLQ_AMT', 'LOAN_MAX_DLQ_AMT']:
    data[column] = data[column].str.replace(',', '.').astype('float')
    


In [6]:
# вновь смотрим типы переменных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15223 entries, 0 to 15222
Data columns (total 52 columns):
AGREEMENT_RK                  15223 non-null object
TARGET                        15223 non-null int64
AGE                           15223 non-null int64
SOCSTATUS_WORK_FL             15223 non-null object
SOCSTATUS_PENS_FL             15223 non-null object
GENDER                        15223 non-null object
CHILD_TOTAL                   15223 non-null int64
DEPENDANTS                    15223 non-null int64
EDUCATION                     15223 non-null object
MARITAL_STATUS                15223 non-null object
GEN_INDUSTRY                  13856 non-null object
GEN_TITLE                     13856 non-null object
ORG_TP_STATE                  13856 non-null object
ORG_TP_FCAPITAL               13858 non-null object
JOB_DIR                       13856 non-null object
FAMILY_INCOME                 15223 non-null object
PERSONAL_INCOME               15223 non-null float64
REG_ADDRES

In [7]:
# проверяем, что запятые заменились на точки
data[['PERSONAL_INCOME','CREDIT', 'FST_PAYMENT', 'LOAN_AVG_DLQ_AMT','LOAN_MAX_DLQ_AMT']].head(10)

,PERSONAL_INCOME,CREDIT,FST_PAYMENT,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT
0,5000.0,8000.00,8650.00,1580.000000,1580.0
1,12000.0,21650.00,4000.00,4020.000000,4020.0
2,9000.0,33126.00,4000.00,0.000000,0.0
3,25000.0,8491.82,5000.00,1589.923333,1590.0
4,12000.0,21990.00,4000.00,1152.150000,2230.0
5,12000.0,30615.00,3500.00,0.000000,0.0
6,15000.0,10450.00,15000.00,0.000000,0.0
7,6000.0,15035.00,465.00,0.000000,0.0
8,100000.0,32914.31,29080.92,0.000000,0.0
9,7000.0,4440.00,540.00,0.000000,0.0


## I.3. Импутация пропусков, не использующая результаты математических вычислений (импутация, которую можно выполнять до/после разбиения на обучение/контроль)

In [8]:
# если в интересующей нас переменной есть пропуск 
# или при этом переменная SOCSTATUS_PENS_FL имеет значение 1,
# заменяем такие пропуски меткой "Не указано"
data['GEN_INDUSTRY'] = np.where(data['GEN_INDUSTRY'].isnull() | (data['SOCSTATUS_PENS_FL'] == 1), 'Не указано', \
                                data['GEN_INDUSTRY'])

data['GEN_TITLE'] = np.where(data['GEN_TITLE'].isnull() | (data['SOCSTATUS_PENS_FL'] == 1), 'Не указано', \
                             data['GEN_TITLE'])

data['ORG_TP_STATE'] = np.where(data['ORG_TP_STATE'].isnull() | (data['SOCSTATUS_PENS_FL'] == 1), 'Не указано', \
                                data['ORG_TP_STATE'])
data['ORG_TP_FCAPITAL'] = np.where(data['ORG_TP_FCAPITAL'].isnull() | (data['SOCSTATUS_PENS_FL'] == 1), 'Не указано', \
                                   data['ORG_TP_FCAPITAL'])

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15223 entries, 0 to 15222
Data columns (total 52 columns):
AGREEMENT_RK                  15223 non-null object
TARGET                        15223 non-null int64
AGE                           15223 non-null int64
SOCSTATUS_WORK_FL             15223 non-null object
SOCSTATUS_PENS_FL             15223 non-null object
GENDER                        15223 non-null object
CHILD_TOTAL                   15223 non-null int64
DEPENDANTS                    15223 non-null int64
EDUCATION                     15223 non-null object
MARITAL_STATUS                15223 non-null object
GEN_INDUSTRY                  15223 non-null object
GEN_TITLE                     15223 non-null object
ORG_TP_STATE                  15223 non-null object
ORG_TP_FCAPITAL               15223 non-null object
JOB_DIR                       13856 non-null object
FAMILY_INCOME                 15223 non-null object
PERSONAL_INCOME               15223 non-null float64
REG_ADDRES

In [9]:
# заменяем пропуски в указанных переменных
# меткой "Не указано"
data['JOB_DIR'] = np.where(data['JOB_DIR'].isnull(), 'Не указано', data['JOB_DIR'])
data['REGION_NM'] = np.where(data['REGION_NM'].isnull(), 'Не указано', data['REGION_NM'])

# пропуски в переменной TP_PROVINCE заменим значением
# переменной FACT_ADDRESS_PROVINCE
data['TP_PROVINCE'] = np.where(data['TP_PROVINCE'].isnull(), 
                               data['FACT_ADDRESS_PROVINCE'], data['TP_PROVINCE'])
    
# заменяем пропуски в переменной 
# PREVIOUS_CARD_NUM_UTILIZED нулями
data['PREVIOUS_CARD_NUM_UTILIZED'] = np.where(data['PREVIOUS_CARD_NUM_UTILIZED'].isnull(), 0, 
                                              data['PREVIOUS_CARD_NUM_UTILIZED'])
# смотрим количество пропусков
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15223 entries, 0 to 15222
Data columns (total 52 columns):
AGREEMENT_RK                  15223 non-null object
TARGET                        15223 non-null int64
AGE                           15223 non-null int64
SOCSTATUS_WORK_FL             15223 non-null object
SOCSTATUS_PENS_FL             15223 non-null object
GENDER                        15223 non-null object
CHILD_TOTAL                   15223 non-null int64
DEPENDANTS                    15223 non-null int64
EDUCATION                     15223 non-null object
MARITAL_STATUS                15223 non-null object
GEN_INDUSTRY                  15223 non-null object
GEN_TITLE                     15223 non-null object
ORG_TP_STATE                  15223 non-null object
ORG_TP_FCAPITAL               15223 non-null object
JOB_DIR                       15223 non-null object
FAMILY_INCOME                 15223 non-null object
PERSONAL_INCOME               15223 non-null float64
REG_ADDRES

In [10]:
data.head(10)

,AGREEMENT_RK,TARGET,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,EDUCATION,MARITAL_STATUS,GEN_INDUSTRY,GEN_TITLE,ORG_TP_STATE,ORG_TP_FCAPITAL,JOB_DIR,FAMILY_INCOME,PERSONAL_INCOME,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,TP_PROVINCE,REGION_NM,REG_FACT_FL,FACT_POST_FL,REG_POST_FL,REG_FACT_POST_FL,REG_FACT_POST_TP_FL,FL_PRESENCE_FL,OWN_AUTO,AUTO_RUS_FL,HS_PRESENCE_FL,COT_PRESENCE_FL,GAR_PRESENCE_FL,LAND_PRESENCE_FL,CREDIT,TERM,FST_PAYMENT,DL_DOCUMENT_FL,GPF_DOCUMENT_FL,FACT_LIVING_TERM,WORK_TIME,FACT_PHONE_FL,REG_PHONE_FL,GEN_PHONE_FL,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_NUM_PAYM,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED
0,59910150,0,49,1,0,1,2,1,Среднее специальное,Состою в браке,Торговля,Рабочий,Частная компания,Без участия,Вспомогательный техперсонал,от 10000 до 20000 руб.,5000.0,Оренбургская область,Оренбургская область,Оренбургская область,Оренбургская область,ПОВОЛЖСКИЙ,1,1,1,1,1,0,0,0,0,0,0,0,8000.00,6,8650.00,0,1,220,18.0,0,0,1,1,1,6,2,1,1580.000000,1580.0,0.0
1,59910230,0,32,1,0,1,3,3,Среднее,Состою в браке,Торговля,Рабочий,Индивидуальный предприниматель,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,12000.0,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,Ставропольский край,ЮЖНЫЙ,1,1,1,1,0,0,0,0,0,0,0,0,21650.00,6,4000.00,0,1,137,97.0,1,0,1,1,1,6,1,1,4020.000000,4020.0,0.0
2,59910525,0,52,1,0,1,4,0,Неполное среднее,Состою в браке,Информационные технологии,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,9000.0,Иркутская область,Иркутская область,Иркутская область,Иркутская область,ВОСТОЧНО-СИБИРСКИЙ,1,1,1,1,1,0,0,0,1,0,0,0,33126.00,12,4000.00,0,1,251,84.0,0,0,1,2,1,11,0,0,0.000000,0.0,0.0
3,59910803,0,39,1,0,1,1,1,Высшее,Состою в браке,Образование,Руководитель среднего звена,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 20000 до 50000 руб.,25000.0,Ростовская область,Ростовская область,Ростовская область,Ростовская область,ЮЖНЫЙ,0,1,0,0,1,1,0,0,0,0,0,0,8491.82,6,5000.00,0,0,36,168.0,1,1,1,1,1,6,3,1,1589.923333,1590.0,0.0
4,59911781,0,30,1,0,0,0,0,Среднее,Состою в браке,Государственная служба,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,12000.0,Кабардино-Балкария,Кабардино-Балкария,Кабардино-Балкария,Ставропольский край,ЮЖНЫЙ,1,1,1,1,0,0,0,0,1,0,0,0,21990.00,12,4000.00,0,1,83,101.0,1,0,1,2,1,16,2,1,1152.150000,2230.0,0.0
5,59911784,0,29,1,0,0,0,0,Среднее,Гражданский брак,Торговля,Специалист,Частная компания,Без участия,Адм-хоз. и трансп. службы,от 20000 до 50000 руб.,12000.0,Белгородская область,Белгородская область,Белгородская область,Белгородская область,ЦЕНТРАЛЬНЫЙ 1,1,1,1,1,1,0,1,0,0,0,0,0,30615.00,12,3500.00,0,0,108,40.0,1,0,1,2,1,11,0,0,0.000000,0.0,0.0
6,59911832,0,35,1,0,0,1,1,Высшее,Разведен(а),Другие сферы,Специалист,Государственная комп./учреж.,Без участия,Участие в основ. деятельности,от 20000 до 50000 руб.,15000.0,Вологодская область,Вологодская область,Вологодская область,Вологодская область,ЦЕНТРАЛЬНЫЙ 2,1,1,1,1,1,1,0,0,0,0,0,0,10450.00,3,15000.00,0,1,48,6.0,0,0,1,2,2,14,0,0,0.000000,0.0,0.0
7,59912034,0,41,1,0,1,0,0,Среднее,Состою в браке,Сельское хозяйство,Рабочий,Частная компания,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,6000.0,Волгоградская область,Волгоградская область,Волгоградская область,Волгоградская область,ЮЖНЫЙ,1,1,1,1,1,1,0,0,0,0,0,0,15035.00,10,465.00,0,1,204,6.0,1,1,1,2,1,11,0,0,0.000000,0.0,0.0
8,59912560,0,53,1,0,0,2,0,Высшее,Состою в браке,Здравоохранение,Руководитель высшего звена,Частная компания,Без участия,Участие в основ. деятельности,свыше 50000 руб.,100000.0,Ярославская область,Ярославская область,Ярославская область,Ярославская область,ЦЕНТРАЛЬНЫЙ 2,1,1,1,1,1,0,0,0,0,0,0,0,32914.31,4,29080.92,0,0,228,156.0,1,0,1,1,1,3,0,0,0.000000,0.0,0.0
9,59912659,0,43,1,0,1,0,0,Среднее специальное,Состою в бр

## I.4. Обработка редких категорий

In [11]:
# выводим статистики по категориальным переменным,
# смотрим unique - количество уникальных значений,
# выявляем бесполезные переменные - переменные, у
# которых уникальных значений столько, сколько
# наблюдений (AGREEMENT_RK), и переменные с одним
# уникальным значением (DL_DOCUMENT_FL)
data.select_dtypes(include='object').describe()


,AGREEMENT_RK,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,EDUCATION,MARITAL_STATUS,GEN_INDUSTRY,GEN_TITLE,ORG_TP_STATE,ORG_TP_FCAPITAL,JOB_DIR,FAMILY_INCOME,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,TP_PROVINCE,REGION_NM,REG_FACT_FL,FACT_POST_FL,REG_POST_FL,REG_FACT_POST_FL,REG_FACT_POST_TP_FL,FL_PRESENCE_FL,OWN_AUTO,AUTO_RUS_FL,HS_PRESENCE_FL,COT_PRESENCE_FL,GAR_PRESENCE_FL,LAND_PRESENCE_FL,DL_DOCUMENT_FL,GPF_DOCUMENT_FL,FACT_PHONE_FL,REG_PHONE_FL,GEN_PHONE_FL
count,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223
unique,15223,2,2,2,7,5,32,13,6,3,11,5,81,81,80,72,12,2,2,2,2,2,2,3,2,2,2,2,2,1,2,2,2,2
top,64585727,1,0,1,Среднее специальное,Состою в браке,Торговля,Специалист,Частная компания,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,Краснодарский край,Краснодарский край,Краснодарский край,Краснодарский край,ЮЖНЫЙ,1,1,1,1,1,0,0,0,0,0,0,0,0,1,1,0,1
freq,1,13847,13176,9964,6518,9416,2303,6680,6267,13009,11452,7077,674,674,674,790,2179,12765,14480,13493,12758,13782,10507,13453,14097,11940,15074,14930,14558,15223,8961,9311,14389,13855


In [12]:
# удаляем идентификационную переменную AGREEMENT_RK,
# потому что у нее количество уникальных значений
# равно количеству наблюдений, переменную DL_DOCUMENT_FL, 
# потому что у нее одно уникальное значение
data.drop(['AGREEMENT_RK', 'DL_DOCUMENT_FL'], axis=1, inplace=True)
data.select_dtypes(include='object').describe()

,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,EDUCATION,MARITAL_STATUS,GEN_INDUSTRY,GEN_TITLE,ORG_TP_STATE,ORG_TP_FCAPITAL,JOB_DIR,FAMILY_INCOME,REG_ADDRESS_PROVINCE,FACT_ADDRESS_PROVINCE,POSTAL_ADDRESS_PROVINCE,TP_PROVINCE,REGION_NM,REG_FACT_FL,FACT_POST_FL,REG_POST_FL,REG_FACT_POST_FL,REG_FACT_POST_TP_FL,FL_PRESENCE_FL,OWN_AUTO,AUTO_RUS_FL,HS_PRESENCE_FL,COT_PRESENCE_FL,GAR_PRESENCE_FL,LAND_PRESENCE_FL,GPF_DOCUMENT_FL,FACT_PHONE_FL,REG_PHONE_FL,GEN_PHONE_FL
count,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223,15223
unique,2,2,2,7,5,32,13,6,3,11,5,81,81,80,72,12,2,2,2,2,2,2,3,2,2,2,2,2,2,2,2,2
top,1,0,1,Среднее специальное,Состою в браке,Торговля,Специалист,Частная компания,Без участия,Участие в основ. деятельности,от 10000 до 20000 руб.,Краснодарский край,Краснодарский край,Краснодарский край,Краснодарский край,ЮЖНЫЙ,1,1,1,1,1,0,0,0,0,0,0,0,1,1,0,1
freq,13847,13176,9964,6518,9416,2303,6680,6267,13009,11452,7077,674,674,674,790,2179,12765,14480,13493,12758,13782,10507,13453,14097,11940,15074,14930,14558,8961,9311,14389,13855


In [13]:
# заново выделим подмножество категориальных переменных
# и пробежим по нему, выводя частоты категорий
# по каждой категориальной переменной, чтобы
# выявить редкие категории
for column in data.select_dtypes(include='object').columns:
    print(data[column].value_counts(dropna=False))

1    13847
0     1376
Name: SOCSTATUS_WORK_FL, dtype: int64
0    13176
1     2047
Name: SOCSTATUS_PENS_FL, dtype: int64
1    9964
0    5259
Name: GENDER, dtype: int64
Среднее специальное               6518
Среднее                           4679
Высшее                            3133
Неоконченное высшее                532
Неполное среднее                   340
Два и более высших образования      20
Ученая степень                       1
Name: EDUCATION, dtype: int64
Состою в браке        9416
Не состоял в браке    3622
Разведен(а)           1245
Вдовец/Вдова           588
Гражданский брак       352
Name: MARITAL_STATUS, dtype: int64
Торговля                                     2303
Не указано                                   2048
Другие сферы                                 1639
Металлургия/Промышленность/Машиностроение    1285
Государственная служба                       1223
Здравоохранение                              1087
Образование                                   923
Транспорт 

In [14]:
# заменяем категорию "Не указано" на категорию "ЮЖНЫЙ"
data['REGION_NM'] = np.where(data['REGION_NM'] == 'Не указано', 'ЮЖНЫЙ', 
                             data['REGION_NM'])
    
# заменяем неверную категорию "ПОВОЛЖСКИЙ" на категорию "ПРИВОЛЖСКИЙ"
data.loc[data['REGION_NM'] == 'ПОВОЛЖСКИЙ', 'REGION_NM'] = 'ПРИВОЛЖСКИЙ'

In [15]:
# укрупняем категории переменной EDUCATION
data.loc[data['EDUCATION'] == 'Ученая степень', 'EDUCATION'] = 'Высшее'
data.loc[data['EDUCATION'] == 'Два и более высших образования', 'EDUCATION'] = 'Высшее'

# смотрим частоты категорий переменной EDUCATION
data['EDUCATION'].value_counts(dropna=False)

Среднее специальное    6518
Среднее                4679
Высшее                 3154
Неоконченное высшее     532
Неполное среднее        340
Name: EDUCATION, dtype: int64

In [16]:
# Переносим из OWN_AUTO значение 2 в 1
data['OWN_AUTO'] = np.where(data['OWN_AUTO'] == 2, 1, data['OWN_AUTO'])

In [17]:
# создаем словарь со строковыми значениями для замены
dct = {'Агинский Бурятский АО': 'Забайкальский край', 
       'Читинская область': 'Забайкальский край',
       'Усть-Ордынский Бурятский АО': 'Иркутская область',
       'Эвенкийский АО': 'Красноярский край'}

# заменяем категории в переменных REG_ADDRESS_PROVINCE, POSTAL_ADDRESS_PROVINCE, 
# FACT_ADDRESS_PROVINCE, TP_PROVINCE
for col in ['REG_ADDRESS_PROVINCE', 'POSTAL_ADDRESS_PROVINCE', 
            'FACT_ADDRESS_PROVINCE', 'TP_PROVINCE']:
    data[col].replace(dct, inplace=True)

In [18]:
# записываем редкие категории переменных REG_ADDRESS_PROVINCE, 
# POSTAL_ADDRESS_PROVINCE, FACT_ADDRESS_PROVINCE в одну 
# отдельную категорию
for col in ['REG_ADDRESS_PROVINCE', 'POSTAL_ADDRESS_PROVINCE', 'FACT_ADDRESS_PROVINCE']:
    data[col] = np.where((data[col] == 'Хакасия')
                       | (data[col] == 'Ямало-Ненецкий АО')
                       | (data[col] == 'Магаданская область')
                       | (data[col] == 'Калмыкия')
                       | (data[col] == 'Дагестан')
                       | (data[col] == 'Чечня'), 
                       'ДРУГОЕ', data[col])

# записываем редкие категории переменной TP_PROVINCE в одну отдельную категорию
data['TP_PROVINCE'] = np.where((data['TP_PROVINCE'] == 'Сахалинская область')
                               | (data['TP_PROVINCE'] == 'Еврейская АО')
                               | (data['TP_PROVINCE'] == 'Магаданская область')
                               | (data['TP_PROVINCE'] == 'Дагестан')
                               | (data['TP_PROVINCE'] == 'Кабардино-Балкария'), 
                               'ДРУГОЕ', data['TP_PROVINCE'])

# записываем некоторые категории переменной JOB_DIR 
# в отдельную категорию
jobdir_lst = ['Реклама и маркетинг', 'Кадровая служба и секретариат',
              'Пр-техн. обесп. и телеком.', 'Юридическая служба']
data.loc[data['JOB_DIR'].isin(jobdir_lst), 'JOB_DIR'] = 'Другое'

In [19]:
# создаем переменную FACT_TP_FL, которая принимает значение 1, если 
# область фактического пребывания клиента и область торговой точки, 
# где клиент брал последний кредит, совпадают, или 0
# в противном случае
data['FACT_TP_FL'] = np.where(data['FACT_ADDRESS_PROVINCE'] == data['TP_PROVINCE'], 
                              1, 0).astype('object')

In [20]:
# значения переменной FACT_LIVING_TERM берем по модулю, 
# чтобы избавиться от отрицательных значений
data['FACT_LIVING_TERM'] = data['FACT_LIVING_TERM'].abs()

In [21]:
a = 0.001
data['PAYMENT'] = (data['CREDIT'] + a ) / (data['TERM'] + a )
data['SHARE_INCOME'] = (data['PERSONAL_INCOME'] +a) / (data['CREDIT'] +a)
data['LOAN_NUM_OPEN'] = data['LOAN_NUM_TOTAL'] - data['LOAN_NUM_CLOSED']
data['PAYM_TO_LOAN'] = (data['LOAN_NUM_PAYM'] +a) / (data['LOAN_NUM_TOTAL'] + a )
data['DLQ_TO_PAYM'] = (data['LOAN_DLQ_NUM'] + a) / (data['LOAN_NUM_PAYM'] + a )

data['FST_SHARE'] = (data['FST_PAYMENT'] +a ) / (data['FST_PAYMENT'] + data['CREDIT'])   

data['PTI'] = (data['PAYMENT'] + a ) / (data['PERSONAL_INCOME'] + a)

In [22]:
# пишем функцию для укрупнения категорий
# переменной REGION_NM в конвейере
def collapse_categories(X):
    dct = {'ЮЖНЫЙ': 'cat1', 
           'ЦЕНТРАЛЬНЫЙ 1': 'cat1', 
           'ЦЕНТРАЛЬНЫЙ 2': 'cat1',
           'ЗАПАДНО-СИБИРСКИЙ': 'cat2', 
           'ВОСТОЧНО-СИБИРСКИЙ': 'cat2', 
           'УРАЛЬСКИЙ': 'cat2',
           'ДАЛЬНЕВОСТОЧНЫЙ': 'cat2', 
           'СЕВЕРО-ЗАПАДНЫЙ': 'cat2', 
           'ЦЕНТРАЛЬНЫЙ ОФИС': 'cat2',
           'ПРИВОЛЖСКИЙ': 'cat3'}
    X.replace(dct, inplace=True)
    return X

In [23]:
# на основе переменной PERSONAL_INCOME создаем переменную 
# PERSONAL_INCOME_CAT, у которой метки категорий будут 
# повторять метки категорий FAMILY_INCOME
bins = [-np.inf, 5000, 10000, 20000, 50000, np.inf]
lab = ['до 5000 руб.', 'от 5000 до 10000 руб.', 'от 10000 до 20000 руб.', 'от 20000 до 50000 руб.', 'свыше 50000 руб.']
data['PERSONAL_INCOME_CAT'] = pd.cut(data['PERSONAL_INCOME'], bins, labels=lab) 

In [24]:
data['PERSONAL_INCOME_CAT'].unique()

[до 5000 руб., от 10000 до 20000 руб., от 5000 до 10000 руб., от 20000 до 50000 руб., свыше 50000 руб.]
Categories (5, object): [до 5000 руб. < от 5000 до 10000 руб. < от 10000 до 20000 руб. < от 20000 до 50000 руб. < свыше 50000 руб.]

In [25]:
data['PERSONAL_FAMILY_INCOME'] = np.where(data['PERSONAL_INCOME_CAT'] == data['FAMILY_INCOME'], 
                                            1, 0).astype('object')
# удаляем переменную PERSONAL_INCOME_CAT
data.drop('PERSONAL_INCOME_CAT', axis=1, inplace=True)

In [26]:
###################################################

In [27]:
# записываем некоторые категории переменной GEN_INDUSTRY 
# в отдельную категорию
data.loc[data['GEN_INDUSTRY'] == 'СМИ/Реклама/PR-агенства', 'GEN_INDUSTRY'] = 'Другие сферы'
data.loc[data['GEN_INDUSTRY'] == 'Юридические услуги/нотариальные услуги',\
         'GEN_INDUSTRY'] = 'Другие сферы'
data.loc[data['GEN_INDUSTRY'] == 'Страхование', 'GEN_INDUSTRY'] = 'Другие сферы'
data.loc[data['GEN_INDUSTRY'] == 'Туризм', 'GEN_INDUSTRY'] = 'Другие сферы'
data.loc[data['GEN_INDUSTRY'] == 'Недвижимость', 'GEN_INDUSTRY'] = 'Другие сферы'
data.loc[data['GEN_INDUSTRY'] == 'Управляющая компания', 'GEN_INDUSTRY'] = 'Другие сферы'
data.loc[data['GEN_INDUSTRY'] == 'Логистика', 'GEN_INDUSTRY'] = 'Другие сферы'
data.loc[data['GEN_INDUSTRY'] == 'Подбор персонала', 'GEN_INDUSTRY'] = 'Другие сферы'
data.loc[data['GEN_INDUSTRY'] == 'Маркетинг', 'GEN_INDUSTRY'] = 'Другие сферы'

# смотрим частоты категорий переменной GEN_INDUSTRY
data['GEN_INDUSTRY'].value_counts(dropna=False)

Торговля                                     2303
Не указано                                   2048
Другие сферы                                 1825
Металлургия/Промышленность/Машиностроение    1285
Государственная служба                       1223
Здравоохранение                              1087
Образование                                   923
Транспорт                                     763
Сельское хозяйство                            670
Строительство                                 556
Коммунальное хоз-во/Дорожные службы           495
Ресторанный бизнес/Общественное питание       393
Наука                                         361
Нефтегазовая промышленность                   220
Банк/Финансы                                  168
Сборочные производства                        167
Энергетика                                    139
Развлечения/Искусство                         129
ЧОП/Детективная д-ть                          128
Информационные услуги                         102


In [28]:
# укрупняем категории переменной GEN_TITLE
data.loc[data['GEN_TITLE'] == 'Партнер', 'GEN_TITLE'] = 'Другое'
data.loc[data['GEN_TITLE'] == 'Военнослужащий по контракту', 'GEN_TITLE'] = 'Другое'

# смотрим частоты категорий переменной GEN_TITLE
data['GEN_TITLE'].value_counts(dropna=False)

Специалист                        6680
Рабочий                           2898
Не указано                        2048
Служащий                           858
Руководитель среднего звена        657
Работник сферы услуг               529
Высококвалифиц. специалист         529
Руководитель высшего звена         409
Другое                             272
Индивидуальный предприниматель     210
Руководитель низшего звена         133
Name: GEN_TITLE, dtype: int64

In [29]:
# укрупняем категории переменной ORG_TP_STATE
data.loc[data['ORG_TP_STATE'] == 'Частная ком. с инос. капиталом', 
         'ORG_TP_STATE'] = 'Частная компания'

# смотрим частоты категорий переменной ORG_TP_STATE
data['ORG_TP_STATE'].value_counts(dropna=False)

Частная компания                  6288
Государственная комп./учреж.      5726
Не указано                        2048
Индивидуальный предприниматель     930
Некоммерческая организация         231
Name: ORG_TP_STATE, dtype: int64

In [30]:
# укрупняем категории переменной JOB_DIR
data.loc[data['JOB_DIR'] == 'Реклама и маркетинг', 'JOB_DIR'] = 'Другое'
data.loc[data['JOB_DIR'] == 'Кадровая служба и секретариат', 'JOB_DIR'] = 'Другое'
data.loc[data['JOB_DIR'] == 'Пр-техн. обесп. и телеком.', 'JOB_DIR'] = 'Другое'
data.loc[data['JOB_DIR'] == 'Юридическая служба', 'JOB_DIR'] = 'Другое'

# смотрим частоты категорий переменной JOB_DIR
data['JOB_DIR'].value_counts(dropna=False)

Участие в основ. деятельности    11452
Не указано                        1367
Вспомогательный техперсонал       1025
Бухгалтерия, финансы, планир.      481
Адм-хоз. и трансп. службы          279
Другое                             238
Снабжение и сбыт                   217
Служба безопасности                164
Name: JOB_DIR, dtype: int64

In [31]:
for column in data.select_dtypes(include='int64').columns:
    print(data[column].value_counts(dropna=False))

0    13411
1     1812
Name: TARGET, dtype: int64
27    485
26    480
33    461
31    461
30    460
34    455
35    442
24    437
32    435
25    430
38    429
29    427
37    420
39    415
28    414
40    406
23    380
36    376
42    373
41    363
44    363
48    358
45    358
50    352
52    349
51    349
49    348
53    339
43    335
56    334
47    320
46    313
54    295
22    292
55    289
57    283
58    276
59    216
61    212
60    206
62    148
63    138
64    127
65     43
66     17
21     13
67      1
Name: AGE, dtype: int64
1     5087
0     4984
2     4140
3      799
4      133
5       59
6       12
7        5
10       3
8        1
Name: CHILD_TOTAL, dtype: int64
0    8226
1    4535
2    2167
3     243
4      42
5       5
6       4
7       1
Name: DEPENDANTS, dtype: int64
6     5063
12    2951
10    2473
3     1919
5      626
9      542
4      446
8      401
18     263
24     166
7      143
11      63
36      50
15      49
14      20
13      17
16      15
20       6
23    

In [32]:
# создаем переменную FACT_TP_FL, которая принимает значение 1, если 
# область фактического пребывания клиента и область торговой точки, 
# где клиент брал последний кредит, совпадают, или 0
# в противном случае
data['FACT_TP_FL'] = np.where(data['FACT_ADDRESS_PROVINCE'] == data['TP_PROVINCE'], 
                              1, 0).astype('object')


In [33]:
data['FACT_TP_FL'].value_counts(dropna=False)

1    14037
0     1186
Name: FACT_TP_FL, dtype: int64

In [34]:
# заменяем категорию "Не указано" на категорию "ЮЖНЫЙ"
data['REGION_NM'] = np.where(data['REGION_NM'] == 'Не указано', 'ЮЖНЫЙ', data['REGION_NM'])
    
# заменяем неверную категорию "ПОВОЛЖСКИЙ" на категорию "ПРИВОЛЖСКИЙ"
data.at[data['REGION_NM'] == 'ПОВОЛЖСКИЙ', 'REGION_NM'] = 'ПРИВОЛЖСКИЙ'

# заменяем Москва на Московский регион
data.at[data['POSTAL_ADDRESS_PROVINCE'] == 'Москва', 'POSTAL_ADDRESS_PROVINCE'] = 'Московская область'
    
# записываем редкие категории в одну отдельную категорию
for i in ['REG_ADDRESS_PROVINCE', 'POSTAL_ADDRESS_PROVINCE', 'FACT_ADDRESS_PROVINCE']:
    data[i] = np.where(#(data[i] == 'Москва') | 
                        (data[i] == 'Хакасия')
                       | (data[i] == 'Ямало-Ненецкий АО')
                       | (data[i] == 'Магаданская область')
                       | (data[i] == 'Калмыкия')
                       | (data[i] == 'Дагестан')
                       | (data[i] == 'Агинский Бурятский АО')
                       | (data[i] == 'Усть-Ордынский Бурятский АО')
                       | (data[i] == 'Эвенкийский АО')
                       | (data[i] == 'Коми-Пермяцкий АО')
                       | (data[i] == 'Чечня'), 
                       'ДРУГОЕ', data[i])

data['TP_PROVINCE'] = np.where((data['TP_PROVINCE'] == 'Сахалинская область')
                               | (data['TP_PROVINCE'] == 'Еврейская АО')
                               | (data['TP_PROVINCE'] == 'Магаданская область')
                               | (data['TP_PROVINCE'] == 'Дагестан')
                               | (data['TP_PROVINCE'] == 'Кабардино-Балкария'), 
                               'ДРУГОЕ', data['TP_PROVINCE'])

# укрупняем категории переменной EDUCATION
data.loc[data['EDUCATION'] == 'Ученая степень', 'EDUCATION'] = 'Высшее'
data.loc[data['EDUCATION'] == 'Два и более высших образования', 'EDUCATION'] = 'Высшее'

# смотрим частоты категорий переменной EDUCATION
data['EDUCATION'].value_counts(dropna=False)



Среднее специальное    6518
Среднее                4679
Высшее                 3154
Неоконченное высшее     532
Неполное среднее        340
Name: EDUCATION, dtype: int64

In [35]:
# отключаем на всякий случай экспоненциальное представление
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# выводим статистики по количественным переменным
data.describe()

,TARGET,AGE,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,CREDIT,TERM,FST_PAYMENT,FACT_LIVING_TERM,WORK_TIME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_NUM_PAYM,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED,PAYMENT,SHARE_INCOME,LOAN_NUM_OPEN,PAYM_TO_LOAN,DLQ_TO_PAYM,FST_SHARE,PTI
count,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,13855.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000
mean,0.119,40.406,1.099,0.645,13853.836,14667.959,8.101,3398.563,3039.343,292.212,1.388,0.752,7.876,0.195,0.130,239.457,243.022,0.019,1906.633,1.408,0.636,5.144,0.025,0.176,0.168
std,0.324,11.601,0.995,0.812,9015.468,12147.873,4.094,5158.109,262455.437,24364.832,0.794,0.988,7.956,0.685,0.356,827.700,836.100,0.138,1390.777,1.271,0.509,2.444,0.080,0.134,1.191
min,0.000,21.000,0.000,0.000,24.000,2000.000,3.000,0.000,0.000,1.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,178.655,0.000,0.000,0.500,0.000,0.000,0.006
25%,0.000,30.000,0.000,0.000,8000.000,6500.000,6.000,1000.000,41.000,24.000,1.000,0.000,4.000,0.000,0.000,0.000,0.000,0.000,1016.497,0.630,0.000,3.997,0.000,0.100,0.085
50%,0.000,39.000,1.000,0.000,12000.000,11550.000,6.000,2000.000,108.000,48.000,1.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,1564.620,1.037,1.000,4.996,0.000,0.125,0.132
75%,0.000,50.000,2.000,1.000,17000.000,19170.000,10.000,4000.000,204.000,110.000,2.000,1.000,8.000,0.000,0.000,0.000,0.000,0.000,2338.869,1.776,1.000,5.995,0.000,0.216,0.203
max,1.000,67.000,10.000,7.000,250000.000,119700.000,36.000,140000.000,28101997.000,2867959.000,11.000,11.000,114.000,13.000,8.000,28012.200,28012.200,2.000,24993.752,31.884,3.000,40.480,0.833,0.871,146.527


In [36]:
# значения переменной FACT_LIVING_TERM берем по модулю, 
# чтобы избавиться от отрицательных значений
data['FACT_LIVING_TERM'] = data['FACT_LIVING_TERM'].abs()
data.describe()

,TARGET,AGE,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,CREDIT,TERM,FST_PAYMENT,FACT_LIVING_TERM,WORK_TIME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_NUM_PAYM,LOAN_DLQ_NUM,LOAN_MAX_DLQ,LOAN_AVG_DLQ_AMT,LOAN_MAX_DLQ_AMT,PREVIOUS_CARD_NUM_UTILIZED,PAYMENT,SHARE_INCOME,LOAN_NUM_OPEN,PAYM_TO_LOAN,DLQ_TO_PAYM,FST_SHARE,PTI
count,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,13855.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000,15223.000
mean,0.119,40.406,1.099,0.645,13853.836,14667.959,8.101,3398.563,3039.343,292.212,1.388,0.752,7.876,0.195,0.130,239.457,243.022,0.019,1906.633,1.408,0.636,5.144,0.025,0.176,0.168
std,0.324,11.601,0.995,0.812,9015.468,12147.873,4.094,5158.109,262455.437,24364.832,0.794,0.988,7.956,0.685,0.356,827.700,836.100,0.138,1390.777,1.271,0.509,2.444,0.080,0.134,1.191
min,0.000,21.000,0.000,0.000,24.000,2000.000,3.000,0.000,0.000,1.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,178.655,0.000,0.000,0.500,0.000,0.000,0.006
25%,0.000,30.000,0.000,0.000,8000.000,6500.000,6.000,1000.000,41.000,24.000,1.000,0.000,4.000,0.000,0.000,0.000,0.000,0.000,1016.497,0.630,0.000,3.997,0.000,0.100,0.085
50%,0.000,39.000,1.000,0.000,12000.000,11550.000,6.000,2000.000,108.000,48.000,1.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,1564.620,1.037,1.000,4.996,0.000,0.125,0.132
75%,0.000,50.000,2.000,1.000,17000.000,19170.000,10.000,4000.000,204.000,110.000,2.000,1.000,8.000,0.000,0.000,0.000,0.000,0.000,2338.869,1.776,1.000,5.995,0.000,0.216,0.203
max,1.000,67.000,10.000,7.000,250000.000,119700.000,36.000,140000.000,28101997.000,2867959.000,11.000,11.000,114.000,13.000,8.000,28012.200,28012.200,2.000,24993.752,31.884,3.000,40.480,0.833,0.871,146.527


In [37]:
data[['FACT_LIVING_TERM', 'WORK_TIME']]

,FACT_LIVING_TERM,WORK_TIME
0,220,18.000
1,137,97.000
2,251,84.000
3,36,168.000
4,83,101.000
...,...,...
15218,121,48.000
15219,300,156.000
15220,48,48.000
15221,270,nan


## I.6. Разбиение на обучающую и контрольную выборки

In [38]:
# создаем обучающий массив признаков, обучающий массив меток,
# тестовый массив признаков, тестовый массив меток
train, test, y_train, y_test = train_test_split(data.drop('TARGET', axis=1), 
                                                data['TARGET'], 
                                                test_size=.3, 
                                                stratify=data['TARGET'], 
                                                random_state=100)

## I.7. Импутация пропусков, использующая статистики - результаты математических вычислений (ее нужно выполнять после разбиения на обучение/контроль)

In [39]:
# наблюдения, в которых количество лет проживания 
# по месту фактического пребывания превышает
# возраст, записываем как пропуски 
train['FACT_LIVING_TERM'] = np.where(train['FACT_LIVING_TERM'] / 12 > train['AGE'], np.NaN, train['FACT_LIVING_TERM'])
test['FACT_LIVING_TERM'] = np.where(test['FACT_LIVING_TERM'] / 12 > test['AGE'], np.NaN, test['FACT_LIVING_TERM'])

# импутируем пропуски медианой, вычисленной 
# на обучающей выборке
train['FACT_LIVING_TERM'].fillna(train['FACT_LIVING_TERM'].median(), inplace=True)
test['FACT_LIVING_TERM'].fillna(train['FACT_LIVING_TERM'].median(), inplace=True)

In [40]:
# наблюдения, в которых время работы в годах превышает 
# возраст (например, человек работает 40 лет, а живет 
# всего 25), записываем как пропуски 
train['WORK_TIME'] = np.where(train['WORK_TIME'] / 12 > train['AGE'], np.NaN, train['WORK_TIME'])
test['WORK_TIME'] = np.where(test['WORK_TIME'] / 12 > test['AGE'], np.NaN, test['WORK_TIME'])

# наблюдения, в которых разница между возрастом и временем работы в годах 
# меньше 16 (например, у 30-летнего время работы в годах составляет 20 лет,
# получается, он работает с 10 лет), записываем как пропуски 

train['WORK_TIME'] = np.where((train['AGE'] - train['WORK_TIME'] / 12) < 16, np.NaN, train['WORK_TIME'])
test['WORK_TIME'] = np.where((test['AGE'] - test['WORK_TIME'] / 12) < 16, np.NaN, test['WORK_TIME'])

# импутируем пропуски медианой, вычисленной 
# на обучающей выборке
train['WORK_TIME'].fillna(train['WORK_TIME'].median(), inplace=True)
test['WORK_TIME'].fillna(train['WORK_TIME'].median(), inplace=True)

In [41]:
# выводим информацию об общем количестве пропущенных 
# наблюдений в обучающей и контрольной выборках
print(train.isnull().sum().sum())
print(test.isnull().sum().sum())

0
0


In [42]:
def importance_auc(train, y_train, test, y_test):
    train_copy = train.copy()
    test_copy = test.copy()
    # создаем список переменных
    col_list = train_copy.select_dtypes(include=['number']).columns
    # создаем список, в который будем записывать auc
    auc_list = []
    for i in col_list:
        train_copy[i].fillna(train_copy[i].median(), inplace=True)
        power = PowerTransformer(
            method='yeo-johnson', standardize=True).fit(train_copy[i].values.reshape(-1,1))
        train_copy[i] = power.transform(train_copy[i].values.reshape(-1,1))
        test_copy[i].fillna(test_copy[i].median(), inplace=True)
        test_copy[i] = power.transform(test_copy[i].values.reshape(-1,1))
        logreg = LogisticRegression(solver='liblinear').fit(train_copy[i].values.reshape(-1,1), y_train)
        auc = roc_auc_score(y_test, logreg.predict_proba(test_copy[i].values.reshape(-1,1))[:, 1])
        auc_list.append(auc)
    
    result = pd.DataFrame({'Переменная': col_list,
                           'AUC': auc_list})
    result = np.round(result.sort_values(by='AUC', ascending=False), 3)
    cm = sns.light_palette('yellow', as_cmap=True)
    return(result.style.background_gradient(cmap=cm))

In [43]:
#importance_auc(train, y_train, test)
importance_auc(train, y_train, test, y_test)

,Переменная,AUC
0,AGE,0.592
3,PERSONAL_INCOME,0.584
23,PTI,0.579
8,WORK_TIME,0.567
7,FACT_LIVING_TERM,0.559
22,FST_SHARE,0.554
5,TERM,0.553
21,DLQ_TO_PAYM,0.55
10,LOAN_NUM_CLOSED,0.549
19,LOAN_NUM_OPEN,0.548


## I.8. Поиск преобразований переменных, максимизирующих нормальность распределения (дается в сокращенном виде)

In [44]:
def diagnostics_skewness(df):
    # создаем список переменных
    col_list = df.select_dtypes(include=['number']).columns
    skew_initial_list = []
    skew_pos_reciprocal_list = []
    skew_neg_reciprocal_list = []
    skew_log_list = []
    skew_corr_log_001_list = []
    skew_corr_log_01_list = []
    skew_corr_log_1_list = []
    skew_corr_log_5_list = []
    skew_cbrt_list = []
    skew_sqrt_list = []
    
    
    # запускаем цикл, который вычисляет скос по каждой 
    # преобразованной переменной
    for i in col_list:
        a = 0.001
        skew_initial = df[i].skew()
        skew_pos_reciprocal = (1 / (df[i].clip(0) + a)).skew()
        skew_neg_reciprocal = (-1 / (df[i].clip(0) + a)).skew()
        skew_log = np.log(df[i].clip(0) + a).skew()
        skew_corr_log_001 = np.log((df[i].clip(0) / df[i].mean()) + 0.001).skew()
        skew_corr_log_01 = np.log((df[i].clip(0) / df[i].mean()) + 0.01).skew()
        skew_corr_log_1 = np.log((df[i].clip(0) / df[i].mean()) + 0.1).skew()
        skew_corr_log_5 = np.log((df[i].clip(0) / df[i].mean()) + 0.5).skew()
        skew_cbrt = np.cbrt(df[i]).skew()
        skew_sqrt = np.sqrt(df[i]).skew()
        skew_initial_list.append(skew_initial)
        skew_pos_reciprocal_list.append(skew_pos_reciprocal)
        skew_neg_reciprocal_list.append(skew_neg_reciprocal)
        skew_log_list.append(skew_log)
        skew_corr_log_001_list.append(skew_corr_log_001)
        skew_corr_log_01_list.append(skew_corr_log_01)
        skew_corr_log_1_list.append(skew_corr_log_1)
        skew_corr_log_5_list.append(skew_corr_log_5)
        skew_cbrt_list.append(skew_cbrt)
        skew_sqrt_list.append(skew_sqrt)   

    result = pd.DataFrame({'Переменная': col_list,
                           'Skew_init': skew_initial_list,
                           'Skew_pos_recip': skew_pos_reciprocal_list,
                           'Skew_neg_recip': skew_neg_reciprocal_list,
                           'Skew_log': skew_log_list,
                           'Skew_adj_log (k=0.001)': skew_corr_log_001_list,
                           'Skew_adj_log (k=0.01)': skew_corr_log_01_list,
                           'Skew_adj_log (k=0.1)': skew_corr_log_1_list,
                           'Skew_adj_log (k=0.5)': skew_corr_log_5_list,
                           'Skew_cbrt': skew_cbrt_list,
                           'Skew_sqrt': skew_sqrt_list})
    result = np.round(result.sort_values(by='Skew_init', ascending=False), 3)
    cm = sns.light_palette('magenta', as_cmap=True)
    return(result.style.background_gradient(cmap=cm))

In [45]:
diagnostics_skewness(train)

,Переменная,Skew_init,Skew_pos_recip,Skew_neg_recip,Skew_log,Skew_adj_log (k=0.001),Skew_adj_log (k=0.01),Skew_adj_log (k=0.1),Skew_adj_log (k=0.5),Skew_cbrt,Skew_sqrt
23,PTI,102.381,2.811,-2.811,-0.012,-0.03,0.003,0.26,0.946,6.33,33.392
16,PREVIOUS_CARD_NUM_UTILIZED,7.211,-7.037,7.037,7.039,7.038,7.038,7.039,7.04,7.046,7.061
12,LOAN_DLQ_NUM,7,-2.269,2.269,2.297,2.287,2.302,2.349,2.459,2.515,2.979
6,FST_PAYMENT,6.744,3.195,-3.195,-2.899,-2.022,-1.195,0.046,1.014,0.186,1.6
14,LOAN_AVG_DLQ_AMT,5.856,-2.269,2.269,2.283,2.301,2.325,2.388,2.511,2.519,2.857
15,LOAN_MAX_DLQ_AMT,5.771,-2.269,2.269,2.282,2.301,2.324,2.387,2.507,2.515,2.845
3,PERSONAL_INCOME,5.024,101.043,-101.043,0.13,0.164,0.253,0.437,0.835,0.941,1.428
21,DLQ_TO_PAYM,4.157,-1.921,1.921,2.36,2.063,2.266,2.412,2.554,2.563,2.835
13,LOAN_MAX_DLQ,3.495,-2.269,2.269,2.271,2.271,2.272,2.275,2.282,2.289,2.337
18,SHARE_INCOME,3.469,100.739,-100.739,0.111,0.119,0.186,0.393,0.839,0.881,1.323


In [46]:
cat_columns = train.dtypes [train.dtypes == 'object'].index.tolist()
num_columns = train.dtypes [train.dtypes != 'object'].index.tolist()

regionnm_regroup = ['REGION_NM']

genindustry_rare = ['GEN_INDUSTRY']
gentitle_rare = ['GEN_TITLE']

#credit_bin = ['CREDIT']
#fstpayment_bin = ['FST_PAYMENT']

jobdir_rare = ['JOB_DIR']
factaddrprov_cntenc = ['FACT_ADDRESS_PROVINCE']

regions_rare = ['REG_ADDRESS_PROVINCE', 'FACT_ADDRESS_PROVINCE', 'POSTAL_ADDRESS_PROVINCE', 'TP_PROVINCE']

In [47]:
# удаляем из списка категориальных переменных
# переменные REGION_NM, GEN_INDUSTRY, GEN_TITLE
cat_columns = list(set(cat_columns).difference(set(regionnm_regroup + 
                                                   factaddrprov_cntenc + genindustry_rare + gentitle_rare + 
                                                   regions_rare + jobdir_rare)))
cat_columns

['GEN_PHONE_FL',
 'EDUCATION',
 'FACT_PHONE_FL',
 'ORG_TP_STATE',
 'GPF_DOCUMENT_FL',
 'REG_POST_FL',
 'GAR_PRESENCE_FL',
 'PERSONAL_FAMILY_INCOME',
 'MARITAL_STATUS',
 'REG_FACT_FL',
 'LAND_PRESENCE_FL',
 'REG_PHONE_FL',
 'OWN_AUTO',
 'REG_FACT_POST_TP_FL',
 'SOCSTATUS_WORK_FL',
 'FAMILY_INCOME',
 'FACT_POST_FL',
 'COT_PRESENCE_FL',
 'REG_FACT_POST_FL',
 'FL_PRESENCE_FL',
 'AUTO_RUS_FL',
 'ORG_TP_FCAPITAL',
 'HS_PRESENCE_FL',
 'SOCSTATUS_PENS_FL',
 'GENDER',
 'FACT_TP_FL']

In [48]:
#  PERSONAL_INCOME - Skew_log
# PTI - Skew_adj_log (k=0.01)
# WORK_TIME - Skew_adj_log(k=0.01)
#  FACT_LIVING_TERM - Skew_adj_log (k=0.5)
#  FST_SHARE - Skew_adj_log (k=0.5)
#  TERM - Skew_adj_log (k=0.1)
#  LOAN_NUM_OPEN - Skew_adj_log (k=0.5)
#  PAYMENT - Skew_log
#  FST_PAYMENT -Skew_adj_log (k=0.1)
#  SHARE_INCOME - Skew_log
#  CREDIT - Skew_log
#  LOAN_NUM_PAYM - Skew_log Skew_adj_log (k=0.001)

# PAYM_TO_LOAN - Skew_cbrt

# LOAN_NUM_TOTAL - 	Skew_pos_recip	Skew_neg_recip
# DEPENDANTS - Skew_pos_recip kew_neg_recip
# DLQ_TO_PAYM - Skew_pos_recip
# LOAN_NUM_CLOSED - Skew_neg_recip
# LOAN_MAX_DLQ_AMT - Skew_pos_recip kew_neg_recip
# LOAN_AVG_DLQ_AMT - Skew_pos_recip	Skew_neg_recip
# LOAN_MAX_DLQ - Skew_pos_recip	Skew_neg_recip
# LOAN_DLQ_NUM - Skew_pos_recip	Skew_neg_recip
# LOAN_MAX_DLQ_AMT - Skew_pos_recip kew_neg_recip
# LOAN_AVG_DLQ_AMT - Skew_pos_recip	Skew_neg_recip
# LOAN_MAX_DLQ - Skew_pos_recip	Skew_neg_recip
# LOAN_DLQ_NUM - Skew_pos_recip	Skew_neg_recip

In [49]:
num_columns_log = ['PERSONAL_INCOME', 'PAYMENT', 'SHARE_INCOME', 'CREDIT', 'LOAN_NUM_PAYM']
num_columns_corrlog0_1 = ['TERM', 'FST_PAYMENT']
num_columns_corrlog0_5 = ['FACT_LIVING_TERM', 'FST_SHARE', 'LOAN_NUM_OPEN']
num_columns_corrlog0_01 = ['PTI', 'WORK_TIME']

num_columns_cbrt = ['PAYM_TO_LOAN'] #

comb_log_lst = num_columns_log + num_columns_corrlog0_1 + num_columns_corrlog0_5 + num_columns_corrlog0_01 

num_columns = list(set(num_columns).difference(set(num_columns_log + num_columns_cbrt)))
num_columns

['PREVIOUS_CARD_NUM_UTILIZED',
 'WORK_TIME',
 'TERM',
 'PTI',
 'LOAN_MAX_DLQ',
 'AGE',
 'LOAN_DLQ_NUM',
 'DLQ_TO_PAYM',
 'DEPENDANTS',
 'FST_PAYMENT',
 'FST_SHARE',
 'LOAN_NUM_CLOSED',
 'LOAN_NUM_OPEN',
 'FACT_LIVING_TERM',
 'LOAN_NUM_TOTAL',
 'CHILD_TOTAL',
 'LOAN_AVG_DLQ_AMT',
 'LOAN_MAX_DLQ_AMT']

In [50]:
# смотрим исходные частоты категорий 
# переменной GEN_TITLE
train['GEN_TITLE'].value_counts(dropna=False)

Специалист                        4660
Рабочий                           2052
Не указано                        1428
Служащий                           591
Руководитель среднего звена        459
Работник сферы услуг               387
Высококвалифиц. специалист         367
Руководитель высшего звена         292
Другое                             176
Индивидуальный предприниматель     148
Руководитель низшего звена          96
Name: GEN_TITLE, dtype: int64

In [51]:
# создаем собственный класс RareGrouper, укрупняющий редкие категории
class RareGrouper(BaseEstimator, TransformerMixin): 
    """
        Параметры:
        - threshold: минимально допустимая относительная частота, при которой замены не происходит
    """
    
    def __init__(self, threshold=20):
        
        # Все очень просто, когда значения представлены целыми числами или строками. 
        # Однако если они представлены в форме списков и словарей, значение (пустой 
        # список или словарь) нужно инициализировать при первом использовании ключа. 
        # defaultdict (дословно: словарь со значением по умолчанию) автоматизирует и 
        # упрощает этот процесс. Он похож на обычный словарь, за исключением одной 
        # особенности - при попытке обратиться к ключу, которого в нем нет, 
        # он сперва добавляет для него значение, используя функцию без аргументов, 
        # которая предоставляется при его создании (в нашем случае речь пойдет о функции 
        # list()).
        # итак, задаем параметры
        self.d = defaultdict(list)
        self.threshold = threshold
        
    def fit(self, X, y=None):
        # для каждой переменной вычисляем абсолютные частоты категорий (calc_col)
        # и получаем словарь вида
        # defaultdict(list,
        #             {'var1': ['категория выше порога', 'категория выше порога'],
        #              'var2': ['категория выше порога', 'категория выше порога']})
        for col in X.columns:
            abs_freq = X[col].value_counts()
            self.d[col] = abs_freq[abs_freq >= self.threshold].index.tolist()
        return self

    def transform(self, X):
        # создаем копию датафрейма
        X = X.copy()
        # для каждой переменной категории (строковые значения), которых нет 
        # в соответствующем списке словаря (их частоты ниже порога), 
        # относим к категории Other
        for col in X.columns:
            X[col] = np.where(X[col].isin(self.d[col]), X[col], 'Other')
        return X

In [52]:
# теперь проверим работу класса RareGrouper для одной переменной,
# категории с количеством менее 70 наблюдений запишем в
# отдельную категорию Other
train_copy = train.copy()
raregrouper = RareGrouper(threshold=70)
raregrouper.fit(train_copy[['GEN_TITLE']])
train_copy['GEN_TITLE'] = raregrouper.transform(train_copy[['GEN_TITLE']])
# смотрим частоты категорий 
# переменной GEN_TITLE
train_copy['GEN_TITLE'].value_counts(dropna=False)

Специалист                        4660
Рабочий                           2052
Не указано                        1428
Служащий                           591
Руководитель среднего звена        459
Работник сферы услуг               387
Высококвалифиц. специалист         367
Руководитель высшего звена         292
Другое                             176
Индивидуальный предприниматель     148
Руководитель низшего звена          96
Name: GEN_TITLE, dtype: int64

In [53]:
# смотрим исходные частоты категорий переменных
# REG_ADDRESS_PROVINCE, POSTAL_ADDRESS_PROVINCE
lst = ['REG_ADDRESS_PROVINCE', 'POSTAL_ADDRESS_PROVINCE']
for col in lst:
    print(train_copy[col].value_counts())

Кемеровская область     481
Краснодарский край      450
Алтайский край          310
Забайкальский край      304
Приморский край         281
                       ... 
Марийская республика     55
Горный Алтай             52
Москва                   46
Еврейская АО             46
Сахалинская область      40
Name: REG_ADDRESS_PROVINCE, Length: 74, dtype: int64
Кемеровская область     481
Краснодарский край      449
Алтайский край          309
Забайкальский край      303
Приморский край         280
                       ... 
Камчатская область       59
Марийская республика     53
Горный Алтай             51
Еврейская АО             48
Сахалинская область      40
Name: POSTAL_ADDRESS_PROVINCE, Length: 73, dtype: int64


In [54]:
# теперь проверим работу класса RareGrouper для списка переменных,
# категории с количеством менее 200 наблюдений запишем в
# отдельную категорию Other
raregroup = RareGrouper(threshold=200)
raregroup.fit(train_copy[lst])
train_copy[lst] = raregroup.transform(train_copy[lst])
# смотрим частоты категорий переменных REG_ADDRESS_PROVINCE, 
# POSTAL_ADDRESS_PROVINCE
for col in lst:
    print(train_copy[col].value_counts())

Other                    7005
Кемеровская область       481
Краснодарский край        450
Алтайский край            310
Забайкальский край        304
Приморский край           281
Ростовская область        274
Свердловская область      239
Саратовская область       239
Воронежская область       218
Омская область            216
Волгоградская область     215
Вологодская область       212
Татарстан                 212
Name: REG_ADDRESS_PROVINCE, dtype: int64
Other                    6805
Кемеровская область       481
Краснодарский край        449
Алтайский край            309
Забайкальский край        303
Приморский край           280
Ростовская область        272
Свердловская область      238
Саратовская область       236
Воронежская область       219
Омская область            215
Волгоградская область     215
Вологодская область       213
Татарстан                 212
Московская область        209
Name: POSTAL_ADDRESS_PROVINCE, dtype: int64


In [55]:
# создаем собственный класс RareGrouper2, укрупняющий редкие категории
class RareGrouper2(TransformerMixin, BaseEstimator):
    """
    Параметры:
    - min_freq: минимально допустимая абсолютная частота, при которой замены не происходит
    - max_features: максимально возможное количество категорий. Если None, то без ограничения
    - include_na: обрабатывать NaN как категорию или игнорировать
    - features: признаки, которые надо обрабатывать:
        - если на вход подается array, то индексы
        - если DataFrame, то список колонок
        - можно подать список колонок для array, но тогда методу .fit() передать fmap
    - ffil_str: чем заменить строковые значения
    - ffil_num: чем заменить числовые значения
    - return_df: должен ли метод .transform() возвращать DataFrame
    """
    def __init__(self,  min_freq=20, max_features=None, include_na=True, features=None, 
                 fill_str ='other', fill_num =-1, return_df=False):
        
        self.features = features
        self.max_features = max_features
        self.min_freq = min_freq
        self.include_na = include_na
        self.fill_str = fill_str
        self.fill_num = fill_num
        self.return_df = return_df
             
    def fit(self, X, y = None, **fit_params):
        
        assert len(X.shape) == 2, 'Array must be 2-dim'
        
        self.fmap = None
        if 'fmap' in fit_params:
            self.fmap = fit_params['fmap']

        X = DataFrame(X, columns = self.fmap)
        if self.features is None:
            features = X.columns
        else:
            features = self.features
        
        self.feats_dict = {}
        for i in features:
            cnt = Counter(X[i])
            if self.include_na:
                keeplist = set([x for x, f in cnt.most_common(self.max_features) if f >= self.min_freq])
            else:
                keeplist = set([x for x, f in cnt.most_common(self.max_features) if f >= self.min_freq \
                                and x is not np.nan])
            
            if np.issubdtype(X[i].dtype, np.number):
                self.feats_dict[i] = (keeplist, self.fill_num)
            else:
                self.feats_dict[i] = (keeplist, self.fill_str)
        return self
    
    def transform(self, X, y = None):
        
        assert len(X.shape) == 2, 'Array must be 2-dim'
        assert 'feats_dict' in self.__dict__, 'Model should be fitted first'
        
        out = DataFrame(X, columns = self.fmap)
        for i in self.feats_dict:
            l, v = self.feats_dict[i]
            out.loc[~out[i].isin(l), i] = v
        if self.return_df:
            return out
        else:
            return out.values

In [56]:
# теперь проверим работу класса RareGrouper2,
# категории с количеством менее 500 наблюдений запишем в
# отдельную категорию other, порог будет равен 500
raregroup2 = RareGrouper2(min_freq=500)
raregroup2.fit(train_copy[['GEN_TITLE']])
train_copy['GEN_TITLE'] = raregroup2.transform(train_copy[['GEN_TITLE']])
train_copy['GEN_TITLE'].value_counts(dropna=False)

Специалист    4660
Рабочий       2052
other         1925
Не указано    1428
Служащий       591
Name: GEN_TITLE, dtype: int64

In [57]:
# создаем собственный класс Replacer, заменяющие отрицательные
# и нулевые значения на небольшие положительные (необходимо
# перед применением логарифмического и обратного преобразований)
class Replacer(BaseEstimator, TransformerMixin):
    """
    Параметры:
        repl: значение для замены
    """
    def __init__(self, repl_value=0.1):
        self.repl_value = repl_value
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_replaced = np.where(X <= 0, self.repl_value, X)
        return X_replaced

In [58]:
# создаем собственный класс CorrLog, выполняюший 
# преобразование log(x/mean(x)+k)
class CorrLog(BaseEstimator, TransformerMixin):
    """
    Параметры:
    lower: нижнее пороговое значение
    a: константа
    k: корректировка асимметрии
    """
    def __init__(self, lower=0.001, k=0.2):
        self.lower = lower
        self.k = k
        self.mean = None
        
    def fit(self, X, y=None):
        self.mean = X.mean()
        return self
    
    def transform(self, X):
        X = np.log(X.clip(self.lower) / self.mean + self.k)
        return X

In [59]:
# пишем функцию, которая проверяет, является ли
# объект массивом NumPy
def is_numpy(x):
    return isinstance(x, np.ndarray)

In [60]:
# создаем собственный класс Replacer, заменяющий 
# категории относительными частотами
class CountEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, min_count=0, nan_value=-1, copy=True):
        self.min_count = min_count
        self.nan_value = nan_value
        self.copy = copy
        self.counts = {}

    def fit(self, x, y=None):
        self.counts = {}
        if len(x.shape) == 1:
            x = x.reshape(-1, 1)
        ncols = x.shape[1]
        is_np = is_numpy(x)
        n_obs = np.float(len(x.index))

        for i in range(ncols):
            if is_np:
                cnt = dict(Counter(x[:, i]))
                cnt = {key: value / n_obs for key, value in cnt.items()}
            else:
                cnt = (x.iloc[:, i].value_counts() / n_obs).to_dict()
            if self.min_count > 0:
                cnt = dict((k, self.nan_value if v < self.min_count else v) for k, v in cnt.items())
            self.counts.update({i: cnt})
        return self

    def transform(self, x):
        if self.copy:
            x = x.copy()
        if len(x.shape) == 1:
            x = x.reshape(-1, 1)
        ncols = x.shape[1]
        is_np = is_numpy(x)

        for i in range(ncols):
            cnt = self.counts[i]
            if is_np:
                k, v = np.array(list(zip(*sorted(cnt.items()))))
                ix = np.digitize(x[:, i], k, right=True)
                x[:, i] = v[ix]
            else:
                x.iloc[:, i].replace(cnt, inplace=True)
        return x

In [61]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [62]:
# создаем конвейер для количественных переменных DLQ_TO_PAYM, 
# LOAN_DLQ_NUM, PREVIOUS_CARD_NUM_UTILIZED, CHILD_TOTAL,
# LOAN_NUM_PAYM, LOAN_NUM_CLOSED, SHARE_INCOME, FST_SHARE,
# DEPENDANTS, AGE, OWN_AUTO, TERM, LOAN_MAX_DLQ, LOAN_NUM_OPEN,
# LOAN_MAX_DLQ_AMT, LOAN_NUM_TOTAL, PAYM_TO_LOAN, к которым
# не будут применены преобразования, максимизирующие нормальность
num_pipe = Pipeline([
    ('scaler', StandardScaler())
])

# создаем конвейер для количественных переменных WORK_TIME, 
# PERSONAL_INCOME, PAYMENT, CREDIT, LOAN_AVG_DLQ_AMT,
# к которым будет применено логарифмическое преобразование
num_log_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='median')),
    ('repl', Replacer(repl_value=0.1)),
    ('log', FunctionTransformer(np.log, validate=False)),
    ('scaler', StandardScaler())
])

# создаем конвейер для количественной переменной,
# к которой применяем логарифмическое преобразование
# log(x/mean(x)+0.5)
num_columns_corrlog0_5_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='median')),
    ('corrlog0_5', CorrLog(k=0.5)),
    ('scaler', StandardScaler())
])

# создаем конвейер для количественной переменной
# к которой применяем логарифмическое преобразование
# log(x/mean(x)+0.1)
num_columns_corrlog0_1_pipe = Pipeline([
    ('corrlog0_1', CorrLog(k=0.1)),
    ('scaler', StandardScaler())
])

# создаем конвейер для количественной переменной,
# к которой применяем логарифмическое преобразование
# log(x/mean(x)+0.5)
num_columns_corrlog0_01_pipe = Pipeline([
    ('corrlog0_01', CorrLog(k=0.01)),
    ('scaler', StandardScaler())
])

# создаем конвейер для категориальной 
# переменной REGION_NM
regionnm_regroup_pipe = Pipeline([
    ('regroup', FunctionTransformer(collapse_categories, validate=False)),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

# создаем конвейер для категориальной 
# переменной REGION_NM
gentitle_rare_pipe = Pipeline([
    ('raregroup', RareGrouper()),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

##
regions_rare_pipe  = Pipeline([
    ('raregroup', RareGrouper()),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

jobdir_rare_pipe  = Pipeline([
    ('raregroup', RareGrouper()),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

##

# создаем конвейер для категориальной 
# переменной REGION_NM
genindustry_rare_pipe = Pipeline([
    ('raregroup2', RareGrouper2()),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

factaddrprov_cntenc_pipe = Pipeline([
    ('raregroup2', RareGrouper2(min_freq=500, return_df=True)),
    ('cntenc', CountEncoder())
])

# создаем конвейер для категориальных переменных
cat_pipe = Pipeline([
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

# создаем список трехэлементных кортежей, в котором
# первый элемент кортежа - название конвейера с
# преобразованиями для определенного типа признаков
transformers = [('num', num_pipe, num_columns),
                ('num_log', num_log_pipe, num_columns_log),
                ('num_columns_corrlog0_5', num_columns_corrlog0_5_pipe, num_columns_corrlog0_5),
                ('num_columns_corrlog0_1', num_columns_corrlog0_1_pipe, num_columns_corrlog0_1),
                ('num_columns_corrlog0_01', num_columns_corrlog0_01_pipe, num_columns_corrlog0_01),
                ('regionnm_regroup', regionnm_regroup_pipe, regionnm_regroup),
                ('factaddrprov_cntenc', factaddrprov_cntenc_pipe, factaddrprov_cntenc),
                ('genindustry_rare', genindustry_rare_pipe, genindustry_rare),
                ('gentitle_rare', gentitle_rare_pipe, gentitle_rare),
                
                ('regions_rare', regions_rare_pipe, regions_rare),
                ('jobdir_rare', jobdir_rare_pipe, jobdir_rare),
                
                ('cat', cat_pipe, cat_columns)]

# передаем список трансформеров в ColumnTransformer
transformer = ColumnTransformer(transformers=transformers)

# задаем итоговый конвейер
pipe = Pipeline([('tf', transformer), 
                 ('logreg', LogisticRegression(solver='liblinear', penalty='l1', random_state=42))])

In [63]:
# задаем сетку гиперпараметров
param_grid = {'tf__gentitle_rare__raregroup__threshold': [20, 300],
              'tf__genindustry_rare__raregroup2__min_freq': [300, 500],
              'logreg__C': [0.14, 0.16, 0.18]}

In [64]:
# создаем экземпляр класса StratifiedKFold, 5-блочную
# стратифицированную перекрестную проверку
strat = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [65]:
# создаем экземпляр класса GridSearchCV, передав конвейер,
# сетку гиперпараметров и указав количество
# блоков перекрестной проверки, отключив запись метрик 
# для обучающих блоков перекрестной проверки в атрибут cv_results_
gs = GridSearchCV(pipe, 
                  param_grid, 
                  scoring='roc_auc',
                  cv=strat, 
                  return_train_score=False)

In [66]:
# выполняем поиск по сетке
gs.fit(train, y_train)
# смотрим наилучшие значения гиперпараметров
print('Наилучшие значения гиперпараметров: {}'.format(gs.best_params_))
# смотрим наилучшее значение AUC
print('Наилучшее значение AUC: {:.3f}'.format(gs.best_score_))
# смотрим значение AUC на тестовой выборке
print('AUC на тестовом наборе: {:.3f}'.format(
    roc_auc_score(y_test, gs.predict_proba(test)[:, 1])))

Наилучшие значения гиперпараметров: {'logreg__C': 0.14, 'tf__genindustry_rare__raregroup2__min_freq': 300, 'tf__gentitle_rare__raregroup__threshold': 300}
Наилучшее значение AUC: 0.707
AUC на тестовом наборе: 0.687
